In [1]:
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt

In [3]:
class FeatureTracking:
    def __init__(self, video_path, output_path, output_fps):
        self.cap = cv2.VideoCapture(video_path)
        self.max_corners = 2000
        self.quality_level = 0.001
        self.min_distance = 0.1

        self.lk_params = dict(winSize=(15, 15), maxLevel=3, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01))

        self.frame_width = 1280
        self.frame_height = 720

        # Region of Interest
        self.square_size = (670, 720)
        self.square_x = int(self.frame_width - self.square_size[0]) // 2  # since the square will be placed relative to the top left corner of the video
        self.square_y = int(self.frame_height - self.square_size[1]) // 2  # we want to make sure that the ROI is placed in the middle
        self.square_position = (self.square_x, self.square_y)

        self.output_path = output_path
        self.output_fps = output_fps

    def process_video(self):
        ret, prev_frame = self.cap.read()
        prev_frame = cv2.resize(prev_frame, (self.frame_width, self.frame_height))  # Resize the frame
        prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        prev_pts = cv2.goodFeaturesToTrack(prev_gray, maxCorners=self.max_corners, qualityLevel=self.quality_level, minDistance=self.min_distance)

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(self.output_path, fourcc, self.output_fps, (self.frame_width, self.frame_height))

        while True:
            for _ in range(3):
                ret, frame = self.cap.read()
                if not ret:
                    break

            # Convert to grayscale
            frame = cv2.resize(frame, (self.frame_width, self.frame_height))
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            # Create ROI
            mask = np.zeros(frame.shape[:2], dtype=np.uint8)
            x, y, w, h = self.square_position[0], self.square_position[1], self.square_size[0], self.square_size[1]
            mask[y:y + h, x:x + w] = 230
            masked_frame = cv2.bitwise_and(frame, frame, mask=mask)

            # Calculate optical flow using Lucas-Kanade on the masked_frame
            prev_frame = cv2.resize(prev_frame, (self.frame_width, self.frame_height))  # Resize the previous frame
            new_pts, status, error = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_pts, None, **self.lk_params)

            valid_new_pts = new_pts[status == 1]
            valid_prev_pts = prev_pts[status == 1]

            for i, (new, prev) in enumerate(zip(valid_new_pts, valid_prev_pts)):
                a, b = new.ravel()
                c, d = prev.ravel()
                a, b, c, d = int(a), int(b), int(c), int(d)

                if self.square_position[0] <= a <= self.square_position[0] + self.square_size[0] and self.square_position[1] <= b <= self.square_position[1] + self.square_size[1]:
                    scaling_factor = 5
                    a_new = int(a - (a - c) * scaling_factor)
                    b_new = int(b - (b - d) * scaling_factor)

                    # Limit drawing to the ROI
                    mask = cv2.line(mask, (a, b), (a_new, b_new), (0, 255, 0), 2)
                    masked_frame = cv2.circle(masked_frame, (a, b), 3, (0, 255, 0), -1)

            result = cv2.add(frame, masked_frame)

            out.write(result)

            cv2.imshow('Frame', result)

            k = cv2.waitKey(30) & 0xff
            if k == 27:
                break

            prev_gray = gray.copy()
            prev_pts = valid_new_pts.reshape(-1, 1, 2)

        self.cap.release()
        out.release()
        cv2.destroyAllWindows()
            
if __name__ == "__main__":
    video_path = "G:\THESIS\SampleVideos\SMTracking_LongSleeve.mp4"
    output_path = "G:\THESIS\LKOF\OutputVideos\output_video.avi"
    output_fps = 1.7 * 30  # 1.5x speedup
    feature_tracker = FeatureTracking(video_path, output_path, output_fps)
    feature_tracker.process_video()


error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


: 